这个notebook的作用是根据合成的vnpy格式的日线主力连续合约, 生成1分钟线的vnpy格式主力连续合约

In [71]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
%matplotlib inline

In [2]:
day_continuous_frame = pd.read_csv(r"D:\data\vnpy\future\day_continuous\MGC.csv")
day_continuous_frame.loc[:, "date"] = pd.to_datetime(day_continuous_frame['date'], format='%Y-%m-%d')
day_continuous_frame = day_continuous_frame.sort_values("date")
day_continuous_frame.loc[:, "weekday"] = day_continuous_frame["date"].apply(lambda x: x.weekday())

In [7]:
one_min_data_paths = glob.glob(r"D:\data\vnpy\future\1_min\*.csv")
one_min_data_paths = [path for path in one_min_data_paths if "MGC" == os.path.basename(path)[0:3]]
one_min_data_paths = sorted(one_min_data_paths)
one_min_data_paths = sorted(one_min_data_paths, key=lambda x: x[-6:-4])

In [16]:
# 需要找到一周每天的每个合约的开盘时间开始和结束. key是周几,value是开盘时间和收盘时间的一个list
# 0是周日
mark_open_time = {
    0: [[(18, 0, 0), (23, 59, 0)]],
    1: [[(0, 0, 0), (17, 0, 0)],
        [(0, 18, 0), (23, 59, 0)]],
    2: [[(0, 0, 0), (17, 0, 0)],
        [(0, 18, 0), (23, 59, 0)]],
    3: [[(0, 0, 0), (17, 0, 0)],
        [(0, 18, 0), (23, 59, 0)]],
    4: [[(0, 0, 0), (17, 0, 0)],
        [(0, 18, 0), (23, 59, 0)]],
    5: [[(0, 0, 0), (17, 0, 0)]],
    6: []
}

In [56]:
#读取并且缓存所有的1min frame, key是symbol, value是data frame
one_min_data_dict = {}
for path in one_min_data_paths:
    symbol = os.path.basename(path)[:-4]
    frame = pd.read_csv(path)
    frame.loc[:, "datetime"] = pd.to_datetime(frame['datetime'], format='%Y%m%d %H:%M:%S')
    frame = frame.sort_values("datetime")
    one_min_data_dict[symbol] = frame

In [26]:
one_min_data_dict.keys()

dict_keys(['MGCZ10', 'MGCG11', 'MGCJ11', 'MGCM11', 'MGCQ11', 'MGCV11', 'MGCZ11', 'MGCG12', 'MGCJ12', 'MGCM12', 'MGCQ12', 'MGCV12', 'MGCZ12', 'MGCG13', 'MGCJ13', 'MGCM13', 'MGCQ13', 'MGCV13', 'MGCZ13', 'MGCG14', 'MGCJ14', 'MGCM14', 'MGCQ14', 'MGCV14', 'MGCZ14', 'MGCG15', 'MGCJ15', 'MGCM15', 'MGCQ15', 'MGCV15', 'MGCZ15', 'MGCG16', 'MGCJ16', 'MGCM16', 'MGCQ16', 'MGCV16', 'MGCZ16', 'MGCG17', 'MGCJ17', 'MGCM17', 'MGCQ17', 'MGCV17', 'MGCZ17', 'MGCG18', 'MGCJ18', 'MGCM18', 'MGCQ18', 'MGCV18', 'MGCZ18', 'MGCG19', 'MGCJ19', 'MGCM19', 'MGCQ19', 'MGCV19', 'MGCZ19', 'MGCG20', 'MGCJ20', 'MGCM20', 'MGCQ20', 'MGCV20', 'MGCZ20'])

In [ ]:
#先建一个空的1min continuous frame
one_min_continuous_frame = pd.DataFrame()
#循环day bar frame
start_time = time.time()
for ix, row in day_continuous_frame.iterrows():
    if ix%50 == 0:
        print("process: {}/{}".format(ix, len(day_continuous_frame)))
        end_time = time.time()
        print("total time used is: {}".format(end_time - start_time))

    date = row.loc["date"]
    weekday = row.loc["weekday"]
    weekday_list = mark_open_time[weekday]
    # 要找到交易时段的所有每一分钟
    for start_end_list in weekday_list:
        min_bar_start_time = datetime(date.year,
                                      date.month,
                                      date.day,
                                      start_end_list[0][0],
                                      start_end_list[0][1],
                                      start_end_list[0][2])
        min_bar_end_time = datetime(date.year,
                                    date.month,
                                    date.day,
                                    start_end_list[1][0],
                                    start_end_list[1][1],
                                    start_end_list[1][2])
        # find all one min timestamps between a start time and an end time
        one_min_time_range = pd.date_range(min_bar_start_time, min_bar_end_time, freq='min')
        # 新建一个one min frame
        new_one_min_frame = pd.DataFrame(columns=day_continuous_frame.columns, index=one_min_time_range)
        # 先把能填充的基础数据都填充了
        symbol = row.loc["symbol"]
        # 找到对应的symbol的实际数据frame
        symbol_frame = one_min_data_dict[symbol]
        # 把symbol data frame 的 index变为找到所有的minute的数据, 其他的数据暂时不要
        new_one_min_frame = symbol_frame.set_index("datetime").reindex(new_one_min_frame.index)
        new_one_min_frame.loc[:, "accumulative_adjust"] = row.loc["accumulative_adjust"]
        price_columns = ["open", "high", "low", "close", "volume"]
        new_one_min_frame.loc[:, price_columns] = new_one_min_frame.loc[:, price_columns] - new_one_min_frame.loc[:, "accumulative_adjust"]
        one_min_continuous_frame = pd.concat([one_min_continuous_frame, new_one_min_frame])
end_time = time.time()
print("total time used is: {}".format(end_time - start_time))

process: 0/3015
total time used is: 0.08895707130432129
process: 50/3015
total time used is: 33.87868595123291
process: 100/3015
total time used is: 68.93140721321106
process: 150/3015
total time used is: 104.83425807952881
